In [4]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

from Model_def import fold_K

# 파일불러오기

In [5]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

df = pd.read_csv('./../0.data/heart_2020_final.csv')
X = df.drop(columns = ['HeartDisease']).values
y = df['HeartDisease']

smote = SMOTE(random_state = 42)
X_smote, y_smote = smote.fit_resample(X,y)

#### catboost 로드

In [13]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(random_state = 42,
                            bagging_temperature = 0,
                            depth = 9,
                            l2_leaf_reg = 3,
                            verbose = 1000,
                            learning_rate =0.1,)
                           # task_type = 'GPU')

#### xgb 로드

In [14]:
xgb = XGBClassifier(random_state = 42,
                    colsample_bytree = 0.8,
                    learning_rate = 0.1,
                    max_depth = 5,
                    max_leaf_nodes = 2,
                    min_child_weight = 4,
                    n_estimators = 200,
                    verbose = 1000,
                    subsample = 0.9,)
                  #  tree_method='gpu_hist')

#### 소프트 보팅 적용

In [15]:
from sklearn.ensemble import VotingClassifier
voting_model = VotingClassifier([('CAT', catboost), ('XGB', xgb)],voting='soft')

fold_K(X_smote, y_smote, voting_model)

0:	learn: 0.5920657	total: 159ms	remaining: 2m 38s
999:	learn: 0.1222221	total: 2m 24s	remaining: 0us
[15:39:57] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_leaf_nodes", "verbose" } are not used.

0:	learn: 0.5919828	total: 148ms	remaining: 2m 27s
999:	learn: 0.1223949	total: 2m 22s	remaining: 0us
[15:43:21] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_leaf_nodes", "verbose" } are not used.

0:	learn: 0.5921759	total: 153ms	remaining: 2m 32s
999:	learn: 0.1221722	total: 2m 25s	remaining: 0us
[15:46:48] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "max_leaf_nodes", "verbose" } are not used.

0:	learn: 0.5926977	total: 124ms	remaining: 2m 3s

VotingClassifier(estimators=[('CAT',
                              <catboost.core.CatBoostClassifier object at 0x0000021425835CD0>),
                             ('XGB',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaf_nodes=2, max_leaves=None,
                                            min_child_weight=4, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, ...))],
                 voting='soft')

In [ ]:
# 보팅 모델 fit
voting_model.fit(X_smote, y_smote)

# 모델 저장
import joblib
xgb.save('/content/last_voting_model.pkl') 